In [ ]:
from pathlib import Path

import numpy as np
from keras.utils import image_dataset_from_directory
from keras.models import Sequential, load_model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications import MobileNetV3Small
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from matplotlib import pyplot as plt
import cv2

In [ ]:
TRAINING_DS_DIR = 'data/output/'
MODEL_PATH = 'data/model/'
OV_MODEL_PATH = 'data/model_ov'
IMG_SIZE = (90, 90)
INPUT_SHAPE = tuple(list(IMG_SIZE) + [3])
BATCH_SIZE = 4
LEARNING_RATE = 0.0001
EPOCHS = 10

MODE = 'LOAD' # LOAD / BUILD
TRAIN = False

## Load dataset

In [ ]:
def image_dataset(subset: str):
    return image_dataset_from_directory(
        TRAINING_DS_DIR,
        validation_split=0.2,
        subset=subset,
        seed=123,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        color_mode='rgb',
        label_mode='binary'
        )

train_ds = image_dataset("training")
val_ds = image_dataset("validation")

## Create model

In [ ]:
if MODE == "BUILD":
    base_model = MobileNetV3Small(
        input_shape=INPUT_SHAPE,
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1, activation="sigmoid")
    ])
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss=BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
else:
    model = load_model(MODEL_PATH)
model.summary()

## Train

In [ ]:
if TRAIN:
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=EPOCHS
    )

    Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)
    model.save(MODEL_PATH)

## Visualize training result

In [ ]:
if 'history' in globals():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(EPOCHS)

    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.show()

    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
! pip install openvino-dev --quiet
! mo --saved_model_dir $MODEL_PATH --output_dir $OV_MODEL_PATH

## Test model

In [ ]:
full_img = cv2.imread("data/val/mark_boxes.jpg", cv2.IMREAD_COLOR)
x_slices = np.linspace(0, full_img.shape[1], 5)
images = [full_img[:, int(x1):int(x2)] for x1, x2 in zip(x_slices[:-1], x_slices[1:])]
images = [cv2.resize(img, dsize=IMG_SIZE) for img in images]

f, ax = plt.subplots(1, 4)
for i, img in enumerate(images):
    ax[i].imshow(img)
    prediction = model.predict(np.array([img]))[0][0]
    ax[i].title.set_text(f"{prediction:4f}")
plt.show()